In [16]:
import math
import requests
import psycopg2

# Connect to the database
conn = psycopg2.connect(
    host='localhost',
    port=5432,
    dbname='postgres',
    user='postgres',
    password='admin'
)

# Create a cursor
cur = conn.cursor()

# Create the table
cur.execute(
    """
    CREATE TABLE vacancies (
        title TEXT,
        description TEXT,
        requirements TEXT,
        responsibilities TEXT,
        salary INTEGER,
        currency TEXT,
        employer TEXT,
        location TEXT,
        industry TEXT,
        specializations TEXT,
        type TEXT,
        url TEXT
    )
    """
)

# Set the base URL and parameters for the request
base_url = 'https://api.hh.ru/vacancies'

# Set the initial values for the pagination variables
page = 0
per_page = 100
found = 10000  # Set the initial value of 'found' to a large number
row = 2  # Start adding data to the second row

# Get a list of all available areas
areas_url = 'https://api.hh.ru/areas'
response = requests.get(areas_url, verify=False)
areas = response.json()

# Loop through the areas
for area in areas:
    # Set the initial page number to 0
    page = 0

    # Set the total number of pages to retrieve
    total_pages = math.ceil(found / per_page)

    # Loop through the pages
    for page in range(total_pages):
        params = {
            'text': 'tableau',
            'area': area['id'],  # Set the area to the current area
            'per_page': str(per_page),
            'page': str(page)
        }

        # Make the API request
        response = requests.get(base_url, params=params, verify=False)

        # Convert the response to JSON
        data = response.json()

        # Check if the 'items' key is empty or if there are no more results
        if not data['items'] or found == 0:
            break

        # Add the data to the database
        for item in data['items']:
            title = item['name']
            description = item.get('description', '')
            requirements = item.get('requirement', '')
            responsibilities = item.get('responsibility', '')
            salary = item['salary'].get('from', 0) if item['salary'] is not None else 0
            currency = item['salary'].get('currency', '') if item['salary'] is not None else ''
            employer = item['employer']['name']
            location = item['area']['name']
            #industry = item['industry']['name']
            #specializations = ', '.join([s['name'] for s in item['specializations']])
            v_type = item['type']['name']
            url = item['alternate_url']

            cur.execute(
                """
                INSERT INTO vacancies (title, description, requirements, responsibilities, salary, currency, employer, location, type, url)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """,
                (title, description, requirements, responsibilities, salary, currency, employer, location, v_type, url)
            )

# Commit the changes to the database
conn.commit()

# Close the cursor and connection
cur.close()
conn.close()


C:\Users\edwar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.hh.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\edwar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.hh.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\edwar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarnin